##### Copyright 2018 The TensorFlow Authors; https://www.apache.org/licenses/LICENSE-2.0, MIT License

# Keras regression model - Predict house prices

<table class="tfo-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/models/blob/master/samples/core/tutorials/keras/basic_regression.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /><span>Run in Google Colab</span></a>  
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/models/blob/master/samples/core/tutorials/keras/basic_regression.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /><span>View source on GitHub</span></a></td></table>

In a *regression* problem, we aim to predict the output of a continuous value, like a price or a probability. Contrast this with a *classification* problem, where we aim to predict a discrete label (for example, where a picture contains an apple or an orange). 

This notebook builds a model to predict the median price of homes in a Boston suburb during the mid-1970s. To do this, we'll provide the model with some data points about the suburb, such as the crime rate and the local property tax rate.

This example uses the `tf.keras` API, see [this guide](https://www.tensorflow.org/guide/keras) for details.

### Notebook setup

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from mymods.lauthom import *

print('TensorFlow version:', tf.__version__)

## The Boston Housing Prices dataset

This [dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) is accessible directly in TensorFlow. Download and shuffle the training set:

In [ ]:
boston_housing = keras.datasets.boston_housing

(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

# Shuffle the training set
order = np.argsort(np.random.random(train_labels.shape))
train_data = train_data[order]
train_labels = train_labels[order]

### Examples and features 

This dataset is much smaller than the others we've worked with so far: it has 506 total examples are split between 404 training examples and 102 test examples:

In [ ]:
print("Training set: {}".format(train_data.shape))  # 404 examples, 13 features
print("Testing set:  {}".format(test_data.shape))   # 102 examples, 13 features

The dataset contains 13 different features:

1.   Per capita crime rate.
2.   Proportion of residential land zoned for lots over 25,000 square feet.
3.   Proportion of non-retail business acres per town.
4.   Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5.   Nitric oxides concentration (parts per 10 million).
6.   Average number of rooms per dwelling.
7.   Proportion of owner-occupied units built prior to 1940.
8.   Weighted distances to five Boston employment centres.
9.   Index of accessibility to radial highways.
10.  Full-value property-tax rate per $10,000.
11.  Pupil-teacher ratio by town.
12.  1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13.  Percentage lower status of the population.

Each one of these input data features is stored using a different scale. Some feature are represented by a proportion between 0 and 1, other features are ranges between 1 and 12, some are ranges between 0 and 100, and so on. This is often the case with real-world data, and understanding how to explore and clean such data is an important skill to develop.

In [ ]:
print(train_data[0])  # Display sample features, notice they different scales

Use the [pandas](https://pandas.pydata.org) library to display the first few rows of the dataset in a nicely formatted table:

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
                'TAX', 'PTRATIO', 'B', 'LSTAT']

df = pd.DataFrame(train_data, columns=column_names)
df.head()

### Labels

The labels are the house prices in thousands of dollars. (You may notice the mid-1970s prices.)

In [ ]:
print(train_labels[0:10])  # Display first 10 entries

## Normalize features

It's recommended to normalize features that use different scales and ranges. For each feature, subtract the mean of the feature and divide by the standard deviation:

In [ ]:
# Test data is *not* used when calculating the mean and std.

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

print(train_data[0])  # First training sample, normalized

Although the model *might* converge without feature normalization, it makes training more difficult, and it makes the resulting model more dependant on the choice of units used in the input.

## Create the model

Let's build our model. Here, we'll use a `Sequential` model with two densely connected hidden layers, and an output later that returns a single, continuous value. The model building steps are wrapped in a function, `build_model`, since we'll create a second model, later on.

In [ ]:
def build_model(img_shape):
    model = keras.Sequential([
        keras.layers.Dense(64, activation=tf.nn.relu, 
                           input_shape=(img_shape, )),  
        keras.layers.Dense(64, activation=tf.nn.relu),
        keras.layers.Dense(1)
    ])

    optimizer = tf.train.RMSPropOptimizer(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae'])
    return model

model = build_model(train_data.shape[1])
model.summary()

## Train the model

The model is trained for 500 epochs, and record the training and validation accuracy in the `history` object.

[Keras Callback API](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/Callback)

In [ ]:
class PrintDot(keras.callbacks.Callback):
    """Display training progress by printing a single dot for each completed epoch.
    """
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('\nEpoch: {}'.format(epoch))
        print('..', end='\b')
    def on_train_begin(self, logs=None):
        print('\nTraining model')
    def on_train_end(self, logs=None):
        print('\nModel is trained\n')

### Capture training history metrics

During training attributes (like; epochs, metrics) are logged and can be retrieved later on.  

###### **Note**: Each training(fit) will start with the previously trained weights. Running below cell will *extend* the training.

In [ ]:
EPOCHS = 500

# Store training stats
history = model.fit(train_data, 
                    train_labels, 
                    epochs=EPOCHS,
                    validation_split=0.2, 
                    verbose=0,
                    callbacks=[PrintDot()])

### Visualize the model's training progress 

We want to use the stats stored in the `history` object to determine how long to train *before* the model stops making progress.

In [ ]:
def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [1000$]')
    plt.plot(history.epoch, np.array(history.history['mean_absolute_error']), 
             label='Train Loss')
    plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
             label = 'Val loss')
    plt.legend()
    plt.ylim([0,5])

plot_history(history)

This graph shows little improvement in the validation loss after about 200 epochs. Training loss will always continue to improve. After the validation loss has converged the models starts to overfit.

### Automatically stop training when the validation score doesn't improve
Let's update the `model.fit` method. We'll use a *callback* that tests a training condition for every epoch. If a set amount of epochs elapses without showing improvement, the training automatically stops.

You can learn more about this callback [here](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping).

In [ ]:
model = build_model(train_data.shape[1])

# The patience parameter is the amount of epochs to check for improvement.
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(train_data, 
                    train_labels, 
                    epochs=EPOCHS,
                    validation_split=0.2, 
                    verbose=0,
                    callbacks=[early_stop, PrintDot()])

plot_history(history)

The graph shows the average error is about \\$2,500 dollars. Is this good? Well, \$2,500 is not an insignificant amount when some of the labels are only $15,000.

### Model performance

In [ ]:
[loss, mae] = model.evaluate(test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: ${:7.2f}".format(mae * 1000))

## Predict

Finally, predict some housing prices using data in the testing set:

In [ ]:
test_predictions = model.predict(test_data).flatten()

print(test_predictions)

## Conclusion

This notebook introduced a few techniques to handle a regresson problem.

* Mean Squared Error (MSE) is a common loss function used for regression problems (different than classification problems).
* Similarly, evaluation metrics used for regression differ from classification. A common regression metric is Mean Absolute Error (MAE).
* When input data features have values with different ranges, each feature should be scaled independently.
* If there is not much training data, prefer a small network with few hidden layers to avoid overfitting.
* Early stopping is a useful technique to prevent overfitting.